# EXERCISE-1

In [39]:
import pandas as pd
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
data=pd.read_csv("C:\\Users\\mca\\Desktop\\ML LAB\\Week 5\\weatherAUS.csv")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


## 1. Preprocess the data, remove the attributes which were are not useful to predict rain. Also, remove rows with at least one missing value for each of them. 

In [40]:
# Remove attributes that are not useful to predict rain (e.g., date, location, etc.)
data = data.drop(['Date', 'Location'], axis=1)

# Remove rows with at least one missing value
data = data.dropna()

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56420 entries, 5939 to 139112
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        56420 non-null  float64
 1   MaxTemp        56420 non-null  float64
 2   Rainfall       56420 non-null  float64
 3   Evaporation    56420 non-null  float64
 4   Sunshine       56420 non-null  float64
 5   WindGustDir    56420 non-null  object 
 6   WindGustSpeed  56420 non-null  float64
 7   WindDir9am     56420 non-null  object 
 8   WindDir3pm     56420 non-null  object 
 9   WindSpeed9am   56420 non-null  float64
 10  WindSpeed3pm   56420 non-null  float64
 11  Humidity9am    56420 non-null  float64
 12  Humidity3pm    56420 non-null  float64
 13  Pressure9am    56420 non-null  float64
 14  Pressure3pm    56420 non-null  float64
 15  Cloud9am       56420 non-null  float64
 16  Cloud3pm       56420 non-null  float64
 17  Temp9am        56420 non-null  float64
 18  Te

In [41]:
data=pd.get_dummies(data,drop_first=True)

## 2. Calculate the Variance Inflation Factor (VIF) value. VIF is a number that determines whether a variable has multicollinearity or not (starts from 1, and it has no upper limit. If the number gets larger, it means the variable has huge multicollinearity on it.).  

In [42]:
# Function to calculate VIF
def calculate_vif(df):
    vif = pd.DataFrame()
    vif["Feature"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif

# Calculate VIF for the dataset
vif_df = calculate_vif(data)
print(vif_df)

             Feature         VIF
0            MinTemp   61.053647
1            MaxTemp  622.942011
2           Rainfall    1.690625
3        Evaporation    7.393829
4           Sunshine   18.096002
..               ...         ...
59      WindDir3pm_W    2.704541
60    WindDir3pm_WNW    2.380500
61    WindDir3pm_WSW    2.625873
62     RainToday_Yes    2.266320
63  RainTomorrow_Yes    2.250375

[64 rows x 2 columns]


## 3. Remove multicollinearities by creating new features. Find the features that have paired values and create the new feature which is the difference value between those pairs.

In [43]:
# Identify pairs of features with high VIF
pairs = [('MinTemp', 'MaxTemp'), ('Temp9am', 'Temp3pm'), ('Humidity9am', 'Humidity3pm')]

# Create new features based on the difference between paired values
for pair in pairs:
    feature1, feature2 = pair
    new_feature = f'{feature1}_{feature2}_diff'
    data[new_feature] = data[feature1] - data[feature2]

# Drop the original features
data = data.drop([pair[0] for pair in pairs] + [pair[1] for pair in pairs], axis=1)

In [44]:
print(data)

        Rainfall  Evaporation  Sunshine  WindGustSpeed  WindSpeed9am  \
5939         0.0         12.0      12.3           48.0           6.0   
5940         0.0         14.8      13.0           37.0          19.0   
5942         0.0         10.8      10.6           46.0          30.0   
5943         0.0         11.4      12.2           31.0           6.0   
5944         0.0         11.2       8.4           35.0          17.0   
...          ...          ...       ...            ...           ...   
139108       0.0          6.0      11.0           35.0           9.0   
139109       0.0          7.6       8.6           37.0          13.0   
139110       0.0          5.6      11.0           33.0          17.0   
139111       0.0          6.2      10.6           26.0           9.0   
139112       0.0          5.6      10.7           30.0          15.0   

        WindSpeed3pm  Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  ...  \
5939            20.0       1006.3       1004.4       2.0    

In [45]:
vif_df = calculate_vif(data)
print(vif_df)

                         Feature        VIF
0                       Rainfall   1.670403
1                    Evaporation   4.698851
2                       Sunshine  16.933358
3                  WindGustSpeed  28.885479
4                   WindSpeed9am   8.420177
..                           ...        ...
56                 RainToday_Yes   2.233233
57              RainTomorrow_Yes   2.168048
58          MinTemp_MaxTemp_diff  24.052583
59          Temp9am_Temp3pm_diff  14.958253
60  Humidity9am_Humidity3pm_diff   6.017485

[61 rows x 2 columns]


## 4. Remove features that have a VIF value above 5. 

In [46]:
# Recalculate VIF after creating new features
vif_df = calculate_vif(data)
print(vif_df)

# Remove features with VIF above 5
features_to_remove = vif_df[vif_df['VIF'] > 5]['Feature']
data = data.drop(features_to_remove, axis=1)

                         Feature        VIF
0                       Rainfall   1.670403
1                    Evaporation   4.698851
2                       Sunshine  16.933358
3                  WindGustSpeed  28.885479
4                   WindSpeed9am   8.420177
..                           ...        ...
56                 RainToday_Yes   2.233233
57              RainTomorrow_Yes   2.168048
58          MinTemp_MaxTemp_diff  24.052583
59          Temp9am_Temp3pm_diff  14.958253
60  Humidity9am_Humidity3pm_diff   6.017485

[61 rows x 2 columns]


## 5. Build a regression model to perform the Rain prediction. Also, tabulate accuracy of the prediction models, before and VIF computation. 

In [52]:
# Split data into features (X) and target (y)
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
X = data.drop('RainTomorrow_Yes', axis=1)  # Adjust column name based on one-hot encoding
y = data['RainTomorrow_Yes']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model before VIF computation
model_before_vif = LogisticRegression(max_iter=1000)
model_before_vif.fit(X_train_scaled, y_train)
y_pred_before_vif = model_before_vif.predict(X_test_scaled)

# Calculate metrics before VIF computation
accuracy_before_vif = accuracy_score(y_test, y_pred_before_vif)
mse_before_vif = mean_squared_error(y_test, y_pred_before_vif)
mae_before_vif = mean_absolute_error(y_test, y_pred_before_vif)
r2_before_vif = r2_score(y_test, y_pred_before_vif)

print(f'Accuracy before VIF computation: {accuracy_before_vif:.2f}')
print(f'MSE before VIF computation: {mse_before_vif:.2f}')
print(f'MAE before VIF computation: {mae_before_vif:.2f}')
print(f'R² before VIF computation: {r2_before_vif:.2f}')

Accuracy before VIF computation: 0.99
MSE before VIF computation: 0.01
MAE before VIF computation: 1.76
R² before VIF computation: 0.96


In [53]:
# Train the model after VIF computation
model_after_vif = LogisticRegression(max_iter=1000)
model_after_vif.fit(X_train_scaled, y_train)
y_pred_after_vif = model_after_vif.predict(X_test_scaled)

# Calculate metrics after VIF computation
accuracy_after_vif = accuracy_score(y_test, y_pred_after_vif)
mse_after_vif = mean_squared_error(y_test, y_pred_after_vif)
mae_after_vif = mean_absolute_error(y_test, y_pred_after_vif)
r2_after_vif = r2_score(y_test, y_pred_after_vif)

print(f'Accuracy after VIF computation: {accuracy_after_vif:.2f}')
print(f'MSE after VIF computation: {mse_after_vif:.2f}')
print(f'MAE after VIF computation: {mae_after_vif:.2f}')
print(f'R² after VIF computation: {r2_after_vif:.2f}')


Accuracy after VIF computation: 0.99
MSE after VIF computation: 0.01
MAE after VIF computation: 1.76
R² after VIF computation: 0.96


### The fact that the performance metrics are identical before and after VIF computation suggests that multicollinearity did not significantly impact the logistic regression model's performance. The model was already highly effective in predicting the target variable, "Rain Tomorrow."